In [1]:
import pandas as pd
import numpy as np
from urllib import urlopen
from geopy.geocoders import Nominatim
import json
import time
geolocator = Nominatim()
train = pd.read_csv('locations.csv', index_col= 0)

In [2]:
def get_level1_locations(x,y):
    url = "http://maps.googleapis.com/maps/api/geocode/json?"
    url += "latlng=%s,%s&sensor=false" % (x, y)
    v = urlopen(url).read()
    j = json.loads(v)
    if len(j['results'])>0:
        components = j['results'][0]['address_components']
        location_list=[]
        for c in components:
            if "neighborhood" in c['types']:
                location_list.append(c['long_name'])
            if "administrative_area_level_2" in c['types']:
                location_list.append(c['long_name'])
            if "postal_code" in c['types']:
                location_list.append(c['long_name'])
        location_list.append(j['results'][0]['formatted_address'])
        return location_list
    return ['', '', '', '']
def formatting(x):
    return x.decode('utf-8', 'ignore').encode('ascii', 'ignore')

In [3]:

locations = []
n = 0
for i in train.index:
    x = get_level1_locations(str(train['latitude'][i]), str(train['longitude'][i]))
    locations.append([train.listing_id[i], formatting(x[0]), formatting(x[1]), formatting(x[2]), formatting(x[3])])
    if n%1000 == 0:
        time.sleep(60)
        print x
    n = n + 1
    
    
    #     try:
#     loc = geolocator.reverse(str(train['latitude'][i]) + ', ' + str(train['longitude'][i]), timeout= 5)
#     loc2 = loc.address.split(',')
#     locations.append([train.listing_id[i], loc2[2], loc2[-5], loc2[-2], loc])
#     except:
#         error.append(i)


['', '', '', '']
['', '', '', '']
['', '', '', '']
['', '', '', '']
['', '', '', '']
['', '', '', '']
['', '', '', '']
['', '', '', '']
['', '', '', '']
['', '', '', '']
['', '', '', '']


KeyboardInterrupt: 

In [7]:
locations = pd.DataFrame(locations)
locations.columns = ['listing_id', 'neighborhood', 'county', 'zipcode', 'address_full']
train = train.join(locations.set_index('listing_id'), on = 'listing_id')
train.to_csv('locations2.csv')